In [2]:
#Imports
import pandas as pd
import numpy as np
from datetime import datetime
import os
import glob
from StockTrakDataReader import portfolio_value
import config

$941,588.48


In [99]:
#Get current date/time
now = datetime.now()
current_time = now.strftime("%m/%d/%Y, %-I:%M %p")
print(current_time)

11/28/2020, 8:14 PM


In [100]:
#Turn portfolio value in integer for calculations
portfolio_value = portfolio_value.replace(',','')
totalportfoliovalue = float(portfolio_value[1:])
print(totalportfoliovalue)

941588.48


In [101]:
#Create message/update
portfolioupdate = 'Dutchmen Alpha Fund Portfolio Update\n\n' + current_time + '\n\n'

In [102]:
#Load in portfolio and view head
datafolder = glob.iglob(config.portfoliodatafolder)
latestportfolio = max(datafolder, key=os.path.getctime)
portfolio = pd.read_csv(latestportfolio, index_col=False)
print(portfolio.head())


  Symbol                  Description  Quantity Currency  LastPrice  \
0    AAL  American Airlines Group Inc      -300      USD      14.98   
1   AAPL                   Apple Inc.      -368      USD     116.59   
2   ABBV                   Abbvie Inc        59      USD     104.89   
3   ADBE   Adobe Systems Incorporated      -100      USD     477.03   
4    ADT                      ADT Inc       759      USD       8.08   

   PricePaid  DayChange ProfitLoss  MarketValue  ProfitLossPercentage  
0      14.92       0.04     -18.00      4494.00                 -0.40  
1     107.99       0.56  -3,164.80     42905.12                 -7.96  
2      86.36       0.69   1,093.27      6188.51                 21.46  
3     460.43       6.95  -1,660.00     47703.00                 -3.61  
4       6.65       0.07   1,085.37      6132.72                 21.50  


In [103]:
#Identify short tickets
shorttickers = portfolio[portfolio['Quantity']<0]

In [104]:
#Identify long tickers
longtickers = portfolio[portfolio['Quantity']>0]

In [105]:
#Identify stocks purchased below the minimum
stockspurchasedbelowmin = portfolio[(portfolio['PricePaid']<5) & (portfolio['Quantity']>0)]

In [106]:
#Identify stocks shorted below the minimum
stocksshortedbelowmin = portfolio[(portfolio['PricePaid']<5) & (portfolio['Quantity']<0)]

In [107]:
#Add the total portfolio value to the update
portfolioupdate += ('The total portfolio value is currently: $' + str(round(totalportfoliovalue,2)) + '\n\n')

In [108]:
#Calculate position weights
portfolio['positionweight'] = portfolio['MarketValue']/totalportfoliovalue

In [109]:
#Identify overweight stocks
overweightstocks = portfolio[portfolio['positionweight']>=.05]
verycloseoverweightstocks = portfolio[(portfolio['positionweight']>=.045) & (portfolio['positionweight']<.05)]
closeoverweightstocks = portfolio[(portfolio['positionweight']>=.04) & (portfolio['positionweight']<.045)]
print(overweightstocks)

   Symbol                 Description  Quantity Currency  LastPrice  \
3    ADBE  Adobe Systems Incorporated      -100      USD     477.03   
6    AMZN            Amazon.com Inc.        -15      USD    3195.34   
80   NFLX               Netflix Inc.        -97      USD     491.36   
83   NVDA          NVIDIA Corporation       -90      USD     530.45   

      PricePaid  DayChange ProfitLoss  MarketValue  ProfitLossPercentage  \
3    460.430000       6.95  -1,660.00     47703.00                 -3.61   
6   3037.500000      10.27  -2,367.60     47930.10                 -5.20   
80   476.295258       6.36  -1,461.28     47661.92                 -3.16   
83   510.301111       1.06  -1,813.40     47740.50                 -3.95   

    positionweight  
3         0.050662  
6         0.050903  
80        0.050619  
83        0.050702  


In [110]:
#Calculate dollar neutrality rate
dollarneutralityratio = longtickers['MarketValue'].sum()/shorttickers['MarketValue'].sum()

In [111]:
#Add short tickers count to update
if len(shorttickers.index) < 40:
    portfolioupdate += ('There are not enough short tickers in the portfolio. There are only ' + str(len(shorttickers)) + ':\n\n' + str(shorttickers[['Symbol','Quantity']])+'\n\n')
else:
    portfolioupdate += ('The number of short tickers is good!. There are: ' + str(len(shorttickers)) + '\n\n')

In [112]:
#Add long tickers count to update
if len(longtickers.index) < 40:
    portfolioupdate += ('There are not enough long tickers in the portfolio. There are only ' + str(len(longtickers)) + ':\n\n'+str(longtickers[['Symbol','Quantity']])+'\n\n')
else:
    portfolioupdate += ('The number of long tickers is good!. There are: ' + str(len(longtickers))+ '\n\n')

In [113]:
#Add stocks purchased below min to update
if len(stockspurchasedbelowmin.index)>0:
    portfolioupdate += ('There are stocks that were purchased below the minimum price for buying of $5:\n\n'+str(stockspurchasedbelowmin[['Symbol','Description','PricePaid']])+'\n\n')
else:
    portfolioupdate += ('No holdings in the portfolio were purchased below the minimum of $5\n\n')

In [114]:
#Add stocks shorted below min to update
if len(stocksshortedbelowmin.index)>0:
    portfolioupdate += ('There are stocks that were shorted below the minimum price for shorting of $5:\n\n'+str(stocksshortedbelowmin[['Symbol','Description','PricePaid']])+'\n\n')
else:
    portfolioupdate += ('No holdings in the portfolio were shorted below the minimum of $5\n\n')

In [115]:
#Add dollar neutrality ratio to update
if dollarneutralityratio < .9 or dollarneutralityratio > 1.1:
    portfolioupdate += ('The portfolio dollar neutrality ratio is out of balance!. Current ratio is: '+str(round(dollarneutralityratio,4))+'\n\n')
else:
    portfolioupdate += ('The portfolio dollar neutrality ratio is in balance!. Current ratio is: ' + str(round(dollarneutralityratio,4))+'\n\n')

In [116]:
#Distribution of porfolio weights
welldistributed = True
if len(overweightstocks.index) > 0:
    portfolioupdate += ('There are overweight stocks with weights over the max of 5%:\n\n'+str(overweightstocks[['Symbol','positionweight']])+"\n\n")
    welldistributed = False
if len(verycloseoverweightstocks.index) > 0:
    portfolioupdate += ('There are stocks very close to overweight with weights over 4.5%:\n\n'+str(verycloseoverweightstocks[['Symbol','positionweight']])+"\n\n")
    welldistributed = False
if len(closeoverweightstocks.index) > 0:
    portfolioupdate += ('There are stocks close to overweight with weights over 4%. Keep an eye on them:\n\n'+str(closeoverweightstocks[['Symbol','positionweight']])+"\n\n")
    welldistributed = False
if welldistributed:
    portfolioupdate += ('Portfolio weights are well-distributed!')


In [117]:
#Define function to return update
def returnportfolioupdate():
    print(portfolioupdate)
returnportfolioupdate()

Dutchmen Alpha Fund Portfolio Update

11/28/2020, 8:14 PM

The total portfolio value is currently: $941588.48

The number of short tickers is good!. There are: 52

The number of long tickers is good!. There are: 63

No holdings in the portfolio were purchased below the minimum of $5

No holdings in the portfolio were shorted below the minimum of $5

The portfolio dollar neutrality ratio is in balance!. Current ratio is: 0.9955

There are overweight stocks with weights over the max of 5%:

   Symbol  positionweight
3    ADBE        0.050662
6    AMZN        0.050903
80   NFLX        0.050619
83   NVDA        0.050702

There are stocks very close to overweight with weights over 4.5%:

    Symbol  positionweight
1     AAPL        0.045567
3     ADBE        0.050662
6     AMZN        0.050903
18     CAT        0.049646
38      FB        0.048682
47    GOOG        0.049515
48   GOOGL        0.047447
62    LULU        0.048507
63    MASI        0.048364
76    MSFT        0.045488
80    NFLX 